# 4. MLflow

Ce notebook enregistre tous les modèles, métriques et le meilleur modèle avec MLflow

## Import des bibliothèques

In [ ]:
import pandas as pd
import numpy as np
import pickle
import mlflow
import mlflow.sklearn
from datetime import datetime
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

print("Bibliothèques importées avec succès")

Bibliothèques importées avec succès


## Configuration de MLflow

In [2]:
# Définir le nom de l'expérience
experiment_name = "Job_Classification_Pipeline"

# Créer ou récupérer l'expérience
mlflow.set_experiment(experiment_name)

# Obtenir l'ID de l'expérience
experiment = mlflow.get_experiment_by_name(experiment_name)
experiment_id = experiment.experiment_id

print(f"Expérience MLflow configurée: {experiment_name}")
print(f"Experiment ID: {experiment_id}")
print(f"Tracking URI: {mlflow.get_tracking_uri()}")

2026/02/16 13:28:04 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.schemas
2026/02/16 13:28:04 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.tables
2026/02/16 13:28:04 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.types
2026/02/16 13:28:04 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.constraints
2026/02/16 13:28:04 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.defaults
2026/02/16 13:28:04 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.comments
2026/02/16 13:28:05 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2026/02/16 13:28:05 INFO mlflow.store.db.utils: Updating database tables
2026/02/16 13:28:05 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2026/02/16 13:28:05 INFO alembic.runtime.migration: Will assume non-transactional DDL.
2026/02/16 13:28:05 INFO alembic.runtime.migration: Running upgrade  -> 451aebb31d03, add metric step
2026/02/16 13:2

Expérience MLflow configurée: Job_Classification_Pipeline
Experiment ID: 1
Tracking URI: sqlite:///mlflow.db


## Chargement des résultats du modeling

In [ ]:
# Charger les résultats du modeling
with open('../data/pkl/modeling_results_gridsearch.pkl', 'rb') as f:
    modeling_results = pickle.load(f)

all_results = modeling_results['all_results']
trained_models = modeling_results['trained_models']
best_model_key = modeling_results['best_model_key']
best_model = modeling_results['best_model']
best_metrics = modeling_results['best_metrics']
label_encoder = modeling_results['label_encoder']

print(f"Résultats chargés avec succès")
print(f"Nombre de modèles entraînés: {len(trained_models)}")
print(f"Meilleur modèle: {best_model_key}")

Résultats chargés avec succès
Nombre de modèles entraînés: 20
Meilleur modèle: Random_Forest_Combined


## Enregistrement de tous les modèles dans MLflow

In [4]:
print("="*80)
print("ENREGISTREMENT DE TOUS LES MODÈLES DANS MLFLOW")
print("="*80)

run_ids = {}
total_models = len(trained_models)
current_model = 0

for model_key, model_info in trained_models.items():
    current_model += 1
    print(f"\n[{current_model}/{total_models}] Enregistrement: {model_key}")
    
    # Extraire le nom du modèle et la configuration des features
    model_name, feature_config = model_key.rsplit('_', 1)
    
    # Démarrer un run MLflow
    with mlflow.start_run(run_name=model_key) as run:
        # Enregistrer les paramètres
        mlflow.log_param("model_type", model_name)
        mlflow.log_param("feature_config", feature_config)
        mlflow.log_param("num_classes", len(label_encoder.classes_))
        
        # Enregistrer les hyperparamètres du modèle
        model_params = model_info['model'].get_params()
        for param_name, param_value in model_params.items():
            # Convertir les valeurs non-sérialisables
            if param_value is None or isinstance(param_value, (int, float, str, bool)):
                mlflow.log_param(f"model_{param_name}", param_value)
        
        # Enregistrer toutes les métriques
        metrics = model_info['metrics']
        mlflow.log_metric("accuracy", metrics['accuracy'])
        mlflow.log_metric("precision_weighted", metrics['precision_weighted'])
        mlflow.log_metric("recall_weighted", metrics['recall_weighted'])
        mlflow.log_metric("f1_weighted", metrics['f1_weighted'])
        mlflow.log_metric("precision_macro", metrics['precision_macro'])
        mlflow.log_metric("recall_macro", metrics['recall_macro'])
        mlflow.log_metric("f1_macro", metrics['f1_macro'])
        mlflow.log_metric("training_time", metrics['training_time'])
        mlflow.log_metric("prediction_time", metrics['prediction_time'])
        
        # Enregistrer le modèle
        mlflow.sklearn.log_model(
            sk_model=model_info['model'],
            artifact_path="model",
            registered_model_name=None  # Ne pas enregistrer dans le Model Registry pour l'instant
        )
        
        # Ajouter un tag pour identifier si c'est le meilleur modèle
        is_best = "yes" if model_key == best_model_key else "no"
        mlflow.set_tag("best_model", is_best)
        mlflow.set_tag("timestamp", datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
        
        # Stocker le run_id
        run_ids[model_key] = run.info.run_id
        
        print(f"   ✓ Run ID: {run.info.run_id}")
        print(f"   ✓ F1-Score: {metrics['f1_weighted']:.4f}")
        print(f"   ✓ Meilleur modèle: {is_best}")

print(f"\n{'='*80}")
print(f"TOUS LES MODÈLES ENREGISTRÉS DANS MLFLOW")
print(f"{'='*80}")

ENREGISTREMENT DE TOUS LES MODÈLES DANS MLFLOW

[1/20] Enregistrement: Logistic_Regression_TF-IDF


2026/02/16 13:28:17 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


   ✓ Run ID: 8a914ae59ee444ce8fc52cba9dbbeac1
   ✓ F1-Score: 0.7329
   ✓ Meilleur modèle: no

[2/20] Enregistrement: Multinomial_NB_TF-IDF


2026/02/16 13:28:48 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


   ✓ Run ID: 7957eed4cb2a46c4bc390bb357e86c08
   ✓ F1-Score: 0.6363
   ✓ Meilleur modèle: no

[3/20] Enregistrement: Linear_SVC_TF-IDF


...

   ✓ Run ID: 13d29faf3ceb4e9abc58b7485b544d32
   ✓ F1-Score: 0.6329
   ✓ Meilleur modèle: no

[19/20] Enregistrement: Random_Forest_Combined


2026/02/16 13:34:08 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


   ✓ Run ID: 040f19c8b1154159a2f5029ce8f464d1
   ✓ F1-Score: 0.7556
   ✓ Meilleur modèle: yes

[20/20] Enregistrement: Decision_Tree_Combined


2026/02/16 13:34:26 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


   ✓ Run ID: 03973d30fc5b45cfafad88e69c795b5a
   ✓ F1-Score: 0.6822
   ✓ Meilleur modèle: no

TOUS LES MODÈLES ENREGISTRÉS DANS MLFLOW


## Enregistrement spécial du meilleur modèle

In [5]:
print("\n" + "="*80)
print("ENREGISTREMENT DU MEILLEUR MODÈLE DANS LE MODEL REGISTRY")
print("="*80)

# Créer un run dédié pour le meilleur modèle
with mlflow.start_run(run_name=f"BEST_MODEL_{best_model_key}") as run:
    # Extraire les informations
    model_name, feature_config = best_model_key.rsplit('_', 1)
    
    # Enregistrer les paramètres
    mlflow.log_param("model_type", model_name)
    mlflow.log_param("feature_config", feature_config)
    mlflow.log_param("num_classes", len(label_encoder.classes_))
    mlflow.log_param("selection_criteria", "f1_weighted")
    
    # Enregistrer les hyperparamètres
    model_params = best_model.get_params()
    for param_name, param_value in model_params.items():
        if param_value is None or isinstance(param_value, (int, float, str, bool)):
            mlflow.log_param(f"model_{param_name}", param_value)
    
    # Enregistrer toutes les métriques
    mlflow.log_metric("accuracy", best_metrics['accuracy'])
    mlflow.log_metric("precision_weighted", best_metrics['precision_weighted'])
    mlflow.log_metric("recall_weighted", best_metrics['recall_weighted'])
    mlflow.log_metric("f1_weighted", best_metrics['f1_weighted'])
    mlflow.log_metric("precision_macro", best_metrics['precision_macro'])
    mlflow.log_metric("recall_macro", best_metrics['recall_macro'])
    mlflow.log_metric("f1_macro", best_metrics['f1_macro'])
    mlflow.log_metric("training_time", best_metrics['training_time'])
    mlflow.log_metric("prediction_time", best_metrics['prediction_time'])
    
    # Enregistrer le modèle dans le Model Registry
    model_uri = mlflow.sklearn.log_model(
        sk_model=best_model,
        artifact_path="best_model",
        registered_model_name="Job_Classification_Best_Model"
    )
    
    # Ajouter des tags
    mlflow.set_tag("best_model", "yes")
    mlflow.set_tag("model_version", "production_candidate")
    mlflow.set_tag("timestamp", datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
    mlflow.set_tag("pipeline_stage", "complete")
    
    best_run_id = run.info.run_id
    
    print(f"\n✓ Meilleur modèle enregistré dans le Model Registry")
    print(f"✓ Run ID: {best_run_id}")
    print(f"✓ Model Name: Job_Classification_Best_Model")
    print(f"✓ Model: {model_name}")
    print(f"✓ Features: {feature_config}")
    print(f"✓ F1-Score: {best_metrics['f1_weighted']:.4f}")
    print(f"✓ Accuracy: {best_metrics['accuracy']:.4f}")


ENREGISTREMENT DU MEILLEUR MODÈLE DANS LE MODEL REGISTRY


2026/02/16 13:34:44 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
Successfully registered model 'Job_Classification_Best_Model'.
Created version '1' of model 'Job_Classification_Best_Model'.



✓ Meilleur modèle enregistré dans le Model Registry
✓ Run ID: 19644568d43f426693a36147e1693345
✓ Model Name: Job_Classification_Best_Model
✓ Model: Random_Forest
✓ Features: Combined
✓ F1-Score: 0.7556
✓ Accuracy: 0.7663


## Enregistrement du tableau comparatif

In [ ]:
# Créer un run pour les métadonnées de l'expérience
with mlflow.start_run(run_name="Experiment_Metadata") as run:
    # Enregistrer le tableau de comparaison comme artifact
    all_results.to_csv('../data/csv/mlflow_comparison.csv', index=False)
    mlflow.log_artifact('../data/csv/mlflow_comparison.csv', artifact_path="comparison")
    
    # Enregistrer des statistiques globales
    mlflow.log_metric("total_models_trained", len(trained_models))
    mlflow.log_metric("best_f1_score", best_metrics['f1_weighted'])
    mlflow.log_metric("best_accuracy", best_metrics['accuracy'])
    mlflow.log_metric("avg_f1_score", all_results['f1_weighted'].mean())
    mlflow.log_metric("std_f1_score", all_results['f1_weighted'].std())
    
    # Tags pour l'expérience
    mlflow.set_tag("experiment_type", "model_comparison")
    mlflow.set_tag("timestamp", datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
    mlflow.set_tag("pipeline_complete", "yes")
    
    print("\n✓ Métadonnées de l'expérience enregistrées")
    print(f"✓ Tableau comparatif enregistré comme artifact")


✓ Métadonnées de l'expérience enregistrées
✓ Tableau comparatif enregistré comme artifact


## Visualisation des runs MLflow

In [7]:
# Récupérer tous les runs de l'expérience
from mlflow.tracking import MlflowClient

client = MlflowClient()
runs = client.search_runs(
    experiment_ids=[experiment_id],
    order_by=["metrics.f1_weighted DESC"]
)

print("\n" + "="*80)
print("TOP 10 RUNS PAR F1-SCORE")
print("="*80)

for i, run in enumerate(runs[:10], 1):
    run_name = run.data.tags.get('mlflow.runName', 'N/A')
    f1_score = run.data.metrics.get('f1_weighted', 0)
    accuracy = run.data.metrics.get('accuracy', 0)
    is_best = run.data.tags.get('best_model', 'no')
    
    best_marker = " ⭐ MEILLEUR" if is_best == "yes" else ""
    
    print(f"{i}. {run_name}{best_marker}")
    print(f"   F1-Score: {f1_score:.4f} | Accuracy: {accuracy:.4f}")
    print(f"   Run ID: {run.info.run_id}")
    print()


TOP 10 RUNS PAR F1-SCORE
1. BEST_MODEL_Random_Forest_Combined ⭐ MEILLEUR
   F1-Score: 0.7556 | Accuracy: 0.7663
   Run ID: 19644568d43f426693a36147e1693345

2. Random_Forest_Combined ⭐ MEILLEUR
   F1-Score: 0.7556 | Accuracy: 0.7663
   Run ID: 040f19c8b1154159a2f5029ce8f464d1

3. Random_Forest_Count
   F1-Score: 0.7510 | Accuracy: 0.7622
   Run ID: c7d9b0443b994a37a2486b3cd1ef99cd

4. Logistic_Regression_Count
   F1-Score: 0.7405 | Accuracy: 0.7419
   Run ID: a648556b46d04685bf9c64d617e15176

5. Random_Forest_TF-IDF
   F1-Score: 0.7397 | Accuracy: 0.7520
   Run ID: 6ef08df9b6c7419fb948f7b7903b0561

6. Logistic_Regression_SVD
   F1-Score: 0.7333 | Accuracy: 0.7337
   Run ID: 6e8891a9c6d44f33bfe5a733d70c5637

7. Logistic_Regression_TF-IDF
   F1-Score: 0.7329 | Accuracy: 0.7337
   Run ID: 8a914ae59ee444ce8fc52cba9dbbeac1

8. Linear_SVC_Count
   F1-Score: 0.7221 | Accuracy: 0.7256
   Run ID: d9135b191253466ea94b0be93f6a49c5

9. Linear_SVC_SVD
   F1-Score: 0.7204 | Accuracy: 0.7276
   Run 

## Statistiques de l'expérience

In [8]:
# Calculer des statistiques
f1_scores = [run.data.metrics.get('f1_weighted', 0) for run in runs]
accuracies = [run.data.metrics.get('accuracy', 0) for run in runs]

print("\n" + "="*80)
print("STATISTIQUES DE L'EXPÉRIENCE")
print("="*80)

print(f"\nNombre total de runs: {len(runs)}")
print(f"\nF1-Score (weighted):")
print(f"  Maximum: {max(f1_scores):.4f}")
print(f"  Minimum: {min(f1_scores):.4f}")
print(f"  Moyenne: {np.mean(f1_scores):.4f}")
print(f"  Écart-type: {np.std(f1_scores):.4f}")

print(f"\nAccuracy:")
print(f"  Maximum: {max(accuracies):.4f}")
print(f"  Minimum: {min(accuracies):.4f}")
print(f"  Moyenne: {np.mean(accuracies):.4f}")
print(f"  Écart-type: {np.std(accuracies):.4f}")


STATISTIQUES DE L'EXPÉRIENCE

Nombre total de runs: 22

F1-Score (weighted):
  Maximum: 0.7556
  Minimum: 0.0000
  Moyenne: 0.6479
  Écart-type: 0.1587

Accuracy:
  Maximum: 0.7663
  Minimum: 0.0000
  Moyenne: 0.6535
  Écart-type: 0.1606


## Informations sur le Model Registry

In [9]:
# Récupérer les informations du modèle enregistré
registered_model_name = "Job_Classification_Best_Model"

try:
    registered_model = client.get_registered_model(registered_model_name)
    latest_versions = client.get_latest_versions(registered_model_name)
    
    print("\n" + "="*80)
    print("INFORMATIONS DU MODEL REGISTRY")
    print("="*80)
    
    print(f"\nNom du modèle: {registered_model.name}")
    print(f"Description: {registered_model.description or 'N/A'}")
    print(f"\nVersions disponibles: {len(latest_versions)}")
    
    for version in latest_versions:
        print(f"\n  Version {version.version}:")
        print(f"    Stage: {version.current_stage}")
        print(f"    Run ID: {version.run_id}")
        print(f"    Status: {version.status}")
        
except Exception as e:
    print(f"Note: {str(e)}")


INFORMATIONS DU MODEL REGISTRY

Nom du modèle: Job_Classification_Best_Model
Description: N/A

Versions disponibles: 1

  Version 1:
    Stage: None
    Run ID: 19644568d43f426693a36147e1693345
    Status: READY


## Sauvegarde des informations MLflow

In [ ]:
# Créer un rapport récapitulatif
mlflow_info = {
    'experiment_name': experiment_name,
    'experiment_id': experiment_id,
    'total_runs': len(runs),
    'best_run_id': best_run_id,
    'best_model_key': best_model_key,
    'best_f1_score': best_metrics['f1_weighted'],
    'best_accuracy': best_metrics['accuracy'],
    'registered_model_name': registered_model_name,
    'all_run_ids': run_ids,
    'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'tracking_uri': mlflow.get_tracking_uri()
}

with open('../data/pkl/mlflow_info.pkl', 'wb') as f:
    pickle.dump(mlflow_info, f)

print("Informations MLflow sauvegardées: ../data/pkl/mlflow_info.pkl")

Informations MLflow sauvegardées: mlflow_info.pkl


## Résumé MLflow

In [ ]:
print("\n" + "="*80)
print("RÉSUMÉ MLFLOW")
print("="*80)

print(f"\n1. EXPÉRIENCE MLFLOW:")
print(f"   - Nom: {experiment_name}")
print(f"   - ID: {experiment_id}")
print(f"   - Tracking URI: {mlflow.get_tracking_uri()}")

print(f"\n2. RUNS ENREGISTRÉS:")
print(f"   - Total: {len(runs)}")
print(f"   - Modèles standards: {len(trained_models)}")
print(f"   - Meilleur modèle: 1")
print(f"   - Métadonnées: 1")

print(f"\n3. MEILLEUR MODÈLE:")
print(f"   - Modèle: {best_model_key}")
print(f"   - Run ID: {best_run_id}")
print(f"   - Registry Name: {registered_model_name}")
print(f"   - F1-Score: {best_metrics['f1_weighted']:.4f}")
print(f"   - Accuracy: {best_metrics['accuracy']:.4f}")

print(f"\n4. MÉTRIQUES ENREGISTRÉES POUR CHAQUE MODÈLE:")
print(f"   - accuracy")
print(f"   - precision_weighted & precision_macro")
print(f"   - recall_weighted & recall_macro")
print(f"   - f1_weighted & f1_macro")
print(f"   - training_time")
print(f"   - prediction_time")

print(f"\n5. ARTIFACTS ENREGISTRÉS:")
print(f"   - Modèles sklearn pour tous les runs")
print(f"   - Tableau de comparaison (CSV)")
print(f"   - Meilleur modèle dans le Model Registry")

print(f"\n6. FICHIERS SAUVEGARDÉS LOCALEMENT:")
print(f"   - ../data/pkl/mlflow_info.pkl")
print(f"   - ../data/csv/mlflow_comparison.csv")

print(f"\n7. ACCÈS À L'INTERFACE MLFLOW:")
print(f"   Commande: mlflow ui")
print(f"   URL: http://localhost:5000")

print("\n" + "="*80)
print("PIPELINE COMPLET TERMINÉ AVEC SUCCÈS")
print("="*80)
print("\nTous les modèles, métriques et le meilleur modèle ont été enregistrés dans MLflow.")
print("Vous pouvez maintenant visualiser les résultats avec 'mlflow ui'.")


RÉSUMÉ MLFLOW

1. EXPÉRIENCE MLFLOW:
   - Nom: Job_Classification_Pipeline
   - ID: 1
   - Tracking URI: sqlite:///mlflow.db

2. RUNS ENREGISTRÉS:
   - Total: 22
   - Modèles standards: 20
   - Meilleur modèle: 1
   - Métadonnées: 1

3. MEILLEUR MODÈLE:
   - Modèle: Random_Forest_Combined
   - Run ID: 19644568d43f426693a36147e1693345
   - Registry Name: Job_Classification_Best_Model
   - F1-Score: 0.7556
   - Accuracy: 0.7663

4. MÉTRIQUES ENREGISTRÉES POUR CHAQUE MODÈLE:
   - accuracy
   - precision_weighted & precision_macro
   - recall_weighted & recall_macro
   - f1_weighted & f1_macro
   - training_time
   - prediction_time

5. ARTIFACTS ENREGISTRÉS:
   - Modèles sklearn pour tous les runs
   - Tableau de comparaison (CSV)
   - Meilleur modèle dans le Model Registry

6. FICHIERS SAUVEGARDÉS LOCALEMENT:
   - mlflow_info.pkl
   - mlflow_comparison.csv

7. ACCÈS À L'INTERFACE MLFLOW:
   Commande: mlflow ui
   URL: http://localhost:5000

PIPELINE COMPLET TERMINÉ AVEC SUCCÈS

Tous les